In [3]:
import numpy as np
import tensorflow as tf
import keras as k
import nibabel as nib

from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = '/content/drive/My Drive/Machine learning/data'

# with open('/content/drive/My Drive/Machine learning/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat /content/drive/My\ Drive/Machine\ learning/foo.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
